In [1]:
# data
import pandas as pd
import numpy as np
import gc
from typing import List
from tqdm import tqdm

# models
from lightgbm import LGBMClassifier, log_evaluation
from sklearn.model_selection import StratifiedGroupKFold

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

In [2]:
def get_amex_metric_calculated(y_true: np.array, y_pred: np.array) -> float:
    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)


def lgb_amex_metric(y_true, y_pred):
    """The competition metric with lightgbm's calling convention"""
    return ('amex', get_amex_metric_calculated(y_true, y_pred), True)

In [3]:
def get_train_data(TRAIN_PATH: str) -> pd.DataFrame:
    '''Returns train dataset'''
    df_train = pd.read_parquet(TRAIN_PATH)
    return df_train


def get_test_data(TEST_PATH: str) -> pd.DataFrame:
    '''Returns test dataset'''
    df_test = pd.read_parquet(TEST_PATH)
    return df_test


def get_target(TARGET_PATH: str) -> pd.DataFrame:
    '''Retruns dataset with train targets'''
    df_train_target = pd.read_csv(TARGET_PATH)
    return df_train_target


def get_train_data_with_target_merged(df_train: pd.DataFrame, df_train_target: pd.DataFrame) -> pd.DataFrame:
    '''Retruns train dataset with target variable merged'''
    df_train_w_target = (
        df_train
        .merge(df_train_target,
            on='customer_ID',
            how='left'
        )
    )
    # df_train_w_target.groupby('target', dropna=False).count()['customer_ID']
    '''
    target
    0    4153582
    1    1377869
    Name: customer_ID, dtype: int64    
    '''
    return df_train_w_target


def get_all_features(df: pd.DataFrame) -> List:
    '''Returns list of all features from the dataset'''
    return list(df)


def get_cat_features() -> List:
    '''Returns list of categorical features from the dataset'''
    cat_features = ['B_30', 'B_38', 'D_114', 
                    'D_116', 'D_117', 'D_120', 
                    'D_126', 'D_63', 'D_64', 
                    'D_66', 'D_68']
    
    return cat_features


def get_num_features(all_features: List, cat_features: List) -> List:
    '''Returns list of all numerical features from the dataset'''
    num_feats = [col for col in all_features if col not in cat_features + ['customer_ID', 'S_2', 'target']]

    return num_feats


def get_df_w_aggrs(df: pd.DataFrame, feats: List) ->  pd.DataFrame:
    '''Returns dataframe with generated aggregates based on numerical features'''

    cid = pd.Categorical(df.pop('customer_ID'), ordered=True)
    last = (cid != np.roll(cid, -1)) # mask for last statement of every customer

    df_min = (df
        .groupby(cid)
        .min()[feats]
        .rename(columns={f: f"{f}_min" for f in feats})
    )
    print(df_min.shape)

    df_max = (df
        .groupby(cid)
        .max()[feats]
        .rename(columns={f: f"{f}_max" for f in feats})
    )
    print(df_max.shape)

    df_avg = (df
        .drop('S_2', axis='columns')
        .groupby(cid)
        .mean()[feats]
        .rename(columns={f: f"{f}_avg" for f in feats})
    )
    print(df_avg.shape)

    df_last = (df
        .loc[last, feats]
        .rename(columns={f: f"{f}_last" for f in feats})
        .set_index(np.asarray(cid[last]))
    )
    print(df_last.shape)

    df_aggrs = (pd.concat([df_min, df_max, df_avg, df_last], axis=1)
        .reset_index()
        .rename(columns={'index': 'customer_ID'})
    )
    print(df_aggrs.shape)

    '''
    del df, df_min, df_max, df_avg, cid, last
    gc.collect()
    '''
    return df_aggrs


def check_zapolnenie(df: pd.DataFrame) -> pd.DataFrame:
    '''Returns pd.DataFrame with isNotNullShare of each column of given df'''
    # Calculate percent of not null share each column 
    col_pct_notNull = [] 
    for col in df.columns: 
        percent_notNull = np.mean(~df[col].isnull())*100 
        col_pct_notNull.append([col, percent_notNull]) 
        
    col_pct_notNull_df = pd.DataFrame(col_pct_notNull, columns = ['column_name','isNotNullShare']).sort_values(by = 'isNotNullShare', ascending = False) 
    #print(col_pct_notNull_df)
    return col_pct_notNull_df

In [16]:
len(num_features)

177

In [17]:
len(cat_features)

11

In [4]:
df_train = get_train_data(TRAIN_PATH='./data/train.parquet')

all_features = get_all_features(df_train)
cat_features = get_cat_features()
num_features = get_num_features(all_features, cat_features)
# len(all_features), len(cat_features), len(num_features) -> (190, 11, 178)

df_train_agg = get_df_w_aggrs(df=df_train, feats=num_features)
df_train_target = get_target(TARGET_PATH='./data/train_labels.csv')
df_train = get_train_data_with_target_merged(df_train=df_train_agg, df_train_target=df_train_target)

'''
df_train.target.value_counts()
target
0    340085
1    118828
Name: count, dtype: int64
'''

# df_test = get_test_data(TEST_PATH='./data/test.parquet')
# df_test = get_df_w_aggrs(df=df_test, feats=num_features)

# zapolnenie_train = check_zapolnenie(df_train)
# zapolnenie_test = check_zapolnenie(df_test)

C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2046887445.py:67: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(cid)


(458913, 177)


C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2046887445.py:74: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(cid)


(458913, 177)


C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2046887445.py:82: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(cid)


(458913, 177)
(458913, 177)
(458913, 709)


'\ndf_train.target.value_counts()\ntarget\n0    340085\n1    118828\nName: count, dtype: int64\n'

### Датасет

In [7]:
df_train = get_train_data(TRAIN_PATH='./data/train.parquet')

In [19]:
df_test = get_test_data(TEST_PATH='./data/test.parquet')

In [13]:
pd.set_option("max_colwidth", 400)

In [15]:
df_train[df_train.customer_ID == "0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a"]

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,0,0,0.0,NaN,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,0,0,0.0,NaN,0,0.005492,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2017-05-28,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,...,-1,-1,-1,0,0,0.0,NaN,0,0.006986,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2017-06-13,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,...,-1,-1,-1,0,0,0.0,NaN,0,0.006527,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2017-07-16,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,...,-1,-1,-1,0,0,0.0,NaN,0,0.008126,0
5,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2017-08-04,0.945964,0,0.007863,1.005006,0.004220,0.110946,0.0,0.009866,...,-1,-1,-1,0,0,0.0,NaN,0,0.002223,0
6,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2017-09-18,0.940705,0,0.018859,1.008024,0.004509,0.103329,0.0,0.000783,...,-1,-1,-1,0,0,0.0,NaN,0,0.007661,0
7,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2017-10-08,0.914767,0,0.014324,1.000242,0.000263,0.108115,0.0,0.007836,...,-1,-1,-1,0,0,0.0,NaN,0,0.009616,0
8,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2017-11-20,0.950845,0,0.016888,1.003995,0.001789,0.102792,0.0,0.009817,...,-1,-1,-1,0,0,0.0,NaN,0,0.004369,0
9,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2017-12-04,0.868580,0,0.001930,1.007504,0.001772,0.100470,0.0,0.006073,...,-1,-1,-1,0,0,0.0,NaN,0,0.004753,0


### PSI

In [20]:
df_train.shape, df_test.shape

((5531451, 190), (11363762, 190))

In [21]:
df_train.P_2.value_counts()

P_2
1.000072    9
1.001606    9
1.002713    9
1.009364    9
1.000193    9
           ..
0.797892    1
0.794033    1
0.794828    1
0.871607    1
0.843141    1
Name: count, Length: 4534509, dtype: int64

In [22]:
def population_stability_index(dev_data, val_data, col_name, num_bins=10):

    bins = np.linspace(dev_data[col_name].min(), dev_data[col_name].max(), num_bins + 1)

    dev_data['bin'] = pd.cut(dev_data[col_name], bins=bins, include_lowest=True)
    val_data['bin'] = pd.cut(val_data[col_name], bins=bins, include_lowest=True)

    dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
    val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')

    merged_counts = dev_group.merge(val_group, on='bin', how='left')

    small_constant = 1e-10
    merged_counts['dev_pct'] = (merged_counts['dev_count'] / len(dev_data)) + small_constant
    merged_counts['val_pct'] = (merged_counts['val_count'] / len(val_data)) + small_constant
    merged_counts['psi'] = (merged_counts['val_pct'] - merged_counts['dev_pct']) * np.log(merged_counts['val_pct'] / merged_counts['dev_pct'])

    return merged_counts['psi'].sum()

In [23]:
psi_value = population_stability_index(df_train, df_test, 'P_2')
print(f"PSI Value: {psi_value}")

PSI Value: 0.0011655577923876564


C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')


In [24]:
len(num_features)

177

In [25]:
len(cat_features)

11

In [26]:
psi_list = []

for col in num_features:
    psi_value = population_stability_index(df_train, df_test, col)
    psi_list.append(psi_value)

C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current

C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current

C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current

C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current

C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current

...

C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current

C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current

C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current

C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current

C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dev_group = dev_data.groupby('bin')[col_name].count().reset_index(name='dev_count')
C:\Users\oobur\AppData\Local\Temp\ipykernel_23872\2360378206.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_group = val_data.groupby('bin')[col_name].count().reset_index(name='val_count')


In [30]:
pd.DataFrame([num_features, psi_list]).T.to_csv("psi.csv") # psi of numerical features

### Однофакторный gini

In [ ]:
'''
clf.fit(
    X: Union[lightgbm.compat.dt_DataTable, List[Union[List[float], List[int]]], numpy.ndarray, pandas.core.frame.DataFrame, scipy.sparse._matrix.spmatrix],
    y: Union[List[float], List[int], numpy.ndarray, pandas.core.series.Series, pandas.core.frame.DataFrame, pyarrow.lib.Array, pyarrow.lib.ChunkedArray],
    sample_weight: Union[List[float], List[int], numpy.ndarray, pandas.core.series.Series, pyarrow.lib.Array, pyarrow.lib.ChunkedArray, NoneType] = None,
    init_score: Union[List[float], List[List[float]], numpy.ndarray, pandas.core.series.Series, pandas.core.frame.DataFrame, pyarrow.lib.Table, pyarrow.lib.Array, pyarrow.lib.ChunkedArray, NoneType] = None,
    eval_set: Optional[List[Tuple[Union[lightgbm.compat.dt_DataTable, List[Union[List[float], List[int]]], numpy.ndarray, pandas.core.frame.DataFrame, scipy.sparse._matrix.spmatrix], Union[List[float], List[int], numpy.ndarray, pandas.core.series.Series, pandas.core.frame.DataFrame, pyarrow.lib.Array, pyarrow.lib.ChunkedArray]]]] = None,
    eval_names: Optional[List[str]] = None,
    eval_sample_weight: Optional[List[Union[List[float], List[int], numpy.ndarray, pandas.core.series.Series, pyarrow.lib.Array, pyarrow.lib.ChunkedArray]]] = None,
    eval_class_weight: Optional[List[float]] = None,
    eval_init_score: Optional[List[Union[List[float], List[List[float]], numpy.ndarray, pandas.core.series.Series, pandas.core.frame.DataFrame, pyarrow.lib.Table, pyarrow.lib.Array, pyarrow.lib.ChunkedArray]]] = None,
    eval_metric: Union[str, Callable[[Optional[numpy.ndarray], numpy.ndarray], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray], List[Tuple[str, float, bool]]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray]], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray]], List[Tuple[str, float, bool]]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray], Optional[numpy.ndarray]], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray], Optional[numpy.ndarray]], List[Tuple[str, float, bool]]], List[Union[str, Callable[[Optional[numpy.ndarray], numpy.ndarray], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray], List[Tuple[str, float, bool]]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray]], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray]], List[Tuple[str, float, bool]]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray], Optional[numpy.ndarray]], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray], Optional[numpy.ndarray]], List[Tuple[str, float, bool]]]]], NoneType] = None,
    feature_name: Union[List[str], ForwardRef("Literal['auto']")] = 'auto',
    categorical_feature: Union[List[str], List[int], ForwardRef("Literal['auto']")] = 'auto',
    callbacks: Optional[List[Callable]] = None,
    init_model: Union[str, pathlib.Path, lightgbm.basic.Booster, lightgbm.sklearn.LGBMModel, NoneType] = None,
) -> 'LGBMClassifier'
'''

In [5]:
# выделение групп признаков (из всех признаков, вариант после аггрегирования)

payment_feats = []
delinq_feats = []
spend_feats = []
balance_feats = []
risk_feats = []

for feat in list(df_train):
    if feat[0] == 'P':
        #print(feat)
        payment_feats.append(feat)
    elif feat[0] == 'D':
        delinq_feats.append(feat)
    elif feat[0] == 'S':
        spend_feats.append(feat)
    elif feat[0] == 'B':
        balance_feats.append(feat)
    elif feat[0] == 'R':
        risk_feats.append(feat)

In [6]:
len(payment_feats) + len(delinq_feats) + len(spend_feats) + len(balance_feats) + len(risk_feats)

708

In [8]:
sgkf_gr = StratifiedGroupKFold(n_splits=5)

In [9]:
odnofactorniy_gini = []

for feat in tqdm((payment_feats + delinq_feats + spend_feats + balance_feats + risk_feats)):
    print("="*50)
    print(feat)
    print("="*50)
    
    score_list = []

    for fold, (idx_tr, idx_va) in enumerate(sgkf_gr.split(X=df_train[['customer_ID', 'target']], y=df_train['target'], groups=df_train['customer_ID'])):
        print(fold)
        X_tr, X_va, y_tr, y_va, clf = None, None, None, None, None


        X_tr = df_train.iloc[idx_tr][[feat]] # 'P_2_min' -> feat
        X_va = df_train.iloc[idx_va][[feat]] # 'P_2_min' -> feat
        y_tr = df_train.target.values[idx_tr]
        y_va = df_train.target.values[idx_va]


        clf = LGBMClassifier(n_estimators=200,
                     max_depth=2,
                     learning_rate=0.05,
                     colsample_bytree=0.8,
                     subsample=0.95,
                     subsample_freq=1,
                     verbosity=-1,
                     random_state=42)

        # abc
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=UserWarning)

            clf.fit(X_tr, 
                    y_tr,
                    eval_set = [(X_va, y_va)],
                    eval_metric=[lgb_amex_metric],
                    callbacks=[log_evaluation(100)]
                   )
        # abc
        X_tr, y_tr = None, None
        y_va_pred = clf.predict_proba(X_va, raw_score=True)

        '''
        try:
            print(len(y_va), len(y_va_pred))
            print(type(y_va), type(y_va_pred))
            print(y_va)
            print()
            print(y_va_pred)

        except:
            print(y_va.shape, y_va_pred.shape)
        '''

        score = get_amex_metric_calculated(y_va, y_va_pred)
        print(f"Score = {score:.5f}")

        score_list.append(score)
        
    odnofactorniy_gini.append(np.mean(score_list))

  0%|                                                                                          | 0/708 [00:00<?, ?it/s]

P_2_min
0
[100]	valid_0's binary_logloss: 0.337375	valid_0's amex: 0.59905
[200]	valid_0's binary_logloss: 0.337074	valid_0's amex: 0.599415
Score = 0.59915
1
[100]	valid_0's binary_logloss: 0.336377	valid_0's amex: 0.599288
[200]	valid_0's binary_logloss: 0.335946	valid_0's amex: 0.599219
Score = 0.59896
2
[100]	valid_0's binary_logloss: 0.338069	valid_0's amex: 0.594974
[200]	valid_0's binary_logloss: 0.337847	valid_0's amex: 0.594606
Score = 0.59437
3
[100]	valid_0's binary_logloss: 0.336796	valid_0's amex: 0.601091
[200]	valid_0's binary_logloss: 0.336457	valid_0's amex: 0.600334
Score = 0.60010
4
[100]	valid_0's binary_logloss: 0.339635	valid_0's amex: 0.597166


  0%|                                                                               | 1/708 [01:10<13:55:49, 70.93s/it]

[200]	valid_0's binary_logloss: 0.339386	valid_0's amex: 0.597362
Score = 0.59710
P_3_min
0
[100]	valid_0's binary_logloss: 0.497178	valid_0's amex: 0.33916
[200]	valid_0's binary_logloss: 0.497014	valid_0's amex: 0.338426
Score = 0.33816
1
[100]	valid_0's binary_logloss: 0.494973	valid_0's amex: 0.346625
[200]	valid_0's binary_logloss: 0.494851	valid_0's amex: 0.346297
Score = 0.34603
2
[100]	valid_0's binary_logloss: 0.497608	valid_0's amex: 0.337203
[200]	valid_0's binary_logloss: 0.497506	valid_0's amex: 0.336747
Score = 0.33650
3
[100]	valid_0's binary_logloss: 0.495076	valid_0's amex: 0.34548
[200]	valid_0's binary_logloss: 0.494925	valid_0's amex: 0.345678
Score = 0.34543
4
[100]	valid_0's binary_logloss: 0.496776	valid_0's amex: 0.342379


  0%|▏                                                                              | 2/708 [02:20<13:44:18, 70.05s/it]

...

[200]	valid_0's binary_logloss: 0.535715	valid_0's amex: 0.200138
Score = 0.19989
R_27_last
0
[100]	valid_0's binary_logloss: 0.520074	valid_0's amex: 0.250482
[200]	valid_0's binary_logloss: 0.519906	valid_0's amex: 0.250973
Score = 0.25073
1
[100]	valid_0's binary_logloss: 0.519869	valid_0's amex: 0.249739
[200]	valid_0's binary_logloss: 0.51974	valid_0's amex: 0.251446
Score = 0.25120
2
[100]	valid_0's binary_logloss: 0.520905	valid_0's amex: 0.247348
[200]	valid_0's binary_logloss: 0.520745	valid_0's amex: 0.24663
Score = 0.24640
3
[100]	valid_0's binary_logloss: 0.519523	valid_0's amex: 0.249782
[200]	valid_0's binary_logloss: 0.519363	valid_0's amex: 0.249549
Score = 0.24932
4
[100]	valid_0's binary_logloss: 0.519799	valid_0's amex: 0.251465


100%|████████████████████████████████████████████████████████████████████████████▉| 707/708 [15:31:47<01:09, 69.11s/it]

[200]	valid_0's binary_logloss: 0.519651	valid_0's amex: 0.249541
Score = 0.24930
R_28_last
0
[100]	valid_0's binary_logloss: 0.571805	valid_0's amex: 0.0190314
[200]	valid_0's binary_logloss: 0.571805	valid_0's amex: 0.0190314
Score = 0.01878
1
[100]	valid_0's binary_logloss: 0.571868	valid_0's amex: 0.0187113
[200]	valid_0's binary_logloss: 0.571869	valid_0's amex: 0.0187113
Score = 0.01846
2
[100]	valid_0's binary_logloss: 0.571782	valid_0's amex: 0.0189725
[200]	valid_0's binary_logloss: 0.571782	valid_0's amex: 0.0189725
Score = 0.01873
3
[100]	valid_0's binary_logloss: 0.571722	valid_0's amex: 0.0194692
[200]	valid_0's binary_logloss: 0.571722	valid_0's amex: 0.0194692
Score = 0.01923
4
[100]	valid_0's binary_logloss: 0.571749	valid_0's amex: 0.0191874


100%|█████████████████████████████████████████████████████████████████████████████| 708/708 [15:32:55<00:00, 79.06s/it]

[200]	valid_0's binary_logloss: 0.571749	valid_0's amex: 0.0191874
Score = 0.01894


In [10]:
pd.DataFrame(odnofactorniy_gini, (payment_feats + delinq_feats + spend_feats + balance_feats + risk_feats), columns=['odnofactorniy_gini']).to_csv("odnofactorniy_gini.csv")

### df.corr

In [31]:
df_train.shape

(5531451, 191)

In [ ]:
corr = df_train.corr()

plt.figure(figsize=(14, 10))
sns.heatmap(corr, anot=True, cmap='coolwarm')

In [32]:
import seaborn

ModuleNotFoundError: No module named 'seaborn'

In [33]:
!pip list

Package                       Version
----------------------------- ---------------
aiobotocore                   2.5.0
aiofiles                      22.1.0
aiohttp                       3.8.5
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
anaconda-anon-usage           0.4.2
anaconda-catalogs             0.2.0
anaconda-client               1.12.1
anaconda-cloud-auth           0.1.3
anaconda-navigator            2.5.0
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8

In [36]:
!pip list

Package                       Version
----------------------------- ---------------
aiobotocore                   2.5.0
aiofiles                      22.1.0
aiohttp                       3.8.5
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
anaconda-anon-usage           0.4.2
anaconda-catalogs             0.2.0
anaconda-client               1.12.1
anaconda-cloud-auth           0.1.3
anaconda-navigator            2.5.0
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8

In [35]:
import seaborn as sns